## SQL exploration

This project aims to be a simple exploration of a database containing facts about countries, like the area of a country, the population, population growth and so on.

We'll do this exploration here in jupyter using `ipython-sql`.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

Let's see what tables we have in this database.

In [2]:
%%sql

SELECT *
  FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Let's have, now, a first look at the facts table, printing its first 5 rows.

In [3]:
%%sql

SELECT *
  FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Population

What is the country with the most/least people in the world; and what about its growth.

In [4]:
%%sql

SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


A bit weird. Let's investigate some more.

In [5]:
%%sql

SELECT *
    FROM facts
    WHERE population IN (SELECT MAX(population) FROM facts)
    OR population IN (SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Ok, so the World and Antarctica are included. But what about the maximum and minimum *without* these two?

In [6]:
%%sql

SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts
WHERE code NOT IN ('ay', 'xx');

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
48,1367485388,0.0,4.02


Let's visualize these countries:

In [7]:
%%sql

SELECT *
    FROM facts
WHERE population IN (SELECT MAX(population) FROM facts WHERE code NOT IN ('ay', 'xx'))
OR population IN (SELECT MIN(population) FROM facts WHERE code NOT IN ('ay', 'xx'))
OR population_growth IN (SELECT MAX(population_growth) FROM facts WHERE code NOT IN ('ay', 'xx'))
OR population_growth IN (SELECT MIN(population_growth) FROM facts WHERE code NOT IN ('ay', 'xx'));

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47
190,vt,Holy See (Vatican City),0,0,0,842,0.0,None,None,None
200,ck,Cocos (Keeling) Islands,14,14,0,596,0.0,None,None,None
207,gl,Greenland,2166086,2166086,None,57733,0.0,14.48,8.49,5.98
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None


### Population growth percentiles

What we see here is that while the maximum and minimum for the population and the maximum for population growth are unique, the minimum for population growth is not unique: a bunch of countries have zero population growth.

What is also surprising is how China's population growth is close to zero. This prompts us to investigate the percentiles of population growth.

In [8]:
%%sql

SELECT name AS 'Country',
    population_growth AS 'Population Growth',
    ROUND(PERCENT_RANK() OVER( ORDER BY population_growth ),2) AS 'Percent Rank'
FROM facts WHERE population_growth <> 'None';


 * sqlite:///factbook.db
Done.


Country,Population Growth,Percent Rank
Holy See (Vatican City),0.0,0.0
Cocos (Keeling) Islands,0.0,0.0
Greenland,0.0,0.0
Pitcairn Islands,0.0,0.0
Greece,0.01,0.02
Norfolk Island,0.01,0.02
Tokelau,0.01,0.02
Falkland Islands (Islas Malvinas),0.01,0.02
Guyana,0.02,0.03
Slovakia,0.02,0.03


For some reason some countries have `None` population_growth -- I suppose its just missing information. So I have to exclude those from the percentile calculation.

With this closer inspection we see that China's population growth is quite typical (if not a bit high) for developed nations.

### Population and death

Let's do some investigation with the `death_rate` column. More specifically, let's see the average and the percentiles.

In [9]:
%%sql

SELECT MIN(death_rate), MAX(death_rate), AVG(death_rate)
    FROM facts;

 * sqlite:///factbook.db
Done.


MIN(death_rate),MAX(death_rate),AVG(death_rate)
1.53,14.89,7.8212719298245625


In [10]:
%%sql

SELECT *
    FROM facts
WHERE death_rate IN (SELECT MAX(death_rate) FROM facts)
OR death_rate IN (SELECT MIN(death_rate) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
98,lt,Lesotho,30355,30355,0,1947701,0.32,25.47,14.89,7.36
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39


In [11]:
%%sql

SELECT AVG(death_rate)
    FROM facts

 * sqlite:///factbook.db
Done.


AVG(death_rate)
7.8212719298245625


Below avarage death rates:

In [12]:
%%sql

SELECT *
    FROM facts
WHERE death_rate <= (SELECT AVG(death_rate) FROM facts)
ORDER BY death_rate;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39
184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36
93,ku,Kuwait,17818,17818,0,2788534,1.62,19.91,2.18,1.58
13,ba,Bahrain,760,760,0,1346613,2.41,13.66,2.69,13.09
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0
241,tk,Turks and Caicos Islands,948,948,0,50280,2.3,16.13,3.1,9.94
151,sa,Saudi Arabia,2149690,2149690,0,27752316,1.46,18.51,3.33,0.55
131,mu,Oman,309500,309500,0,3286936,2.07,24.44,3.36,0.43
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05
254,we,West Bank,5860,5640,220,2785366,1.95,22.99,3.5,0.0


In [13]:
%%sql

SELECT *
    FROM facts
WHERE death_rate > (SELECT AVG(death_rate) FROM facts)
ORDER BY death_rate DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
98,lt,Lesotho,30355,30355,0,1947701,0.32,25.47,14.89,7.36
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
71,pu,Guinea-Bissau,36125,28120,8005,1726170,1.91,33.38,14.33,0.0
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26
35,cd,Chad,None,1259200,24800,11631456,1.89,36.6,14.28,3.45
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27
122,wa,Namibia,824292,823290,1002,2212307,0.59,19.8,13.91,0.0
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
34,ct,Central African Republic,622984,622984,0,5391539,2.13,35.08,13.8,0.0


These are certainly intriguing statistics -- I'd not have guessed this order.

Now, for greater detail let's see the percentiles:

In [14]:
%%sql

SELECT name 'Country',
    death_rate 'Death Rate',
    ROUND(PERCENT_RANK() OVER( ORDER BY death_rate ), 2) AS 'Percent Rank'
FROM facts
WHERE death_rate <> 'None';

 * sqlite:///factbook.db
Done.


Country,Death Rate,Percent Rank
Qatar,1.53,0.0
United Arab Emirates,1.97,0.0
Kuwait,2.18,0.01
Bahrain,2.69,0.01
Gaza Strip,3.04,0.02
Turks and Caicos Islands,3.1,0.02
Saudi Arabia,3.33,0.03
Oman,3.36,0.03
Singapore,3.43,0.04
West Bank,3.5,0.04


The list below are countries that have a higher death rate than a birth rate.

In [15]:
%%sql

SELECT name Country, birth_rate 'Birth Rate', death_rate 'Death Rate'
    FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate DESC;

 * sqlite:///factbook.db
Done.


Country,Birth Rate,Death Rate
Ukraine,10.72,14.46
Bulgaria,8.92,14.44
Latvia,10.0,14.31
Lithuania,10.1,14.27
Russia,11.6,13.69
Serbia,9.08,13.66
Belarus,10.7,13.36
Hungary,9.16,12.73
Moldova,12.0,12.59
Estonia,10.51,12.4


## Density

Finally, for this initial exploration let's see the most densely populated countries.

In [16]:
%%sql

SELECT name Country, ROUND(CAST(population AS Float)/CAST(area AS Float),2) Density
    FROM facts
WHERE Density <> 'None'
ORDER BY Density DESC;

 * sqlite:///factbook.db
Done.


Country,Density
Macau,21168.96
Monaco,15267.5
Singapore,8141.28
Hong Kong,6445.04
Gaza Strip,5191.82
Gibraltar,4876.33
Bahrain,1771.86
Maldives,1319.64
Malta,1310.02
Bermuda,1299.93


But to better grasp this information let's calculate the avarage density:

In [17]:
%%sql

SELECT AVG(CAST(population AS Float)/CAST(area AS Float)) 'Avarage Density'
    FROM facts;

 * sqlite:///factbook.db
Done.


Avarage Density
419.66252469247945


We can see clearly, just from the above, that the density is highly rightly skewed. That is, there are few countries with very high density that elevates the mean; but, really, the majority are below the mean.

In [18]:
%%sql

SELECT COUNT(*) 'Countries with density below or avarage'
    FROM facts
WHERE CAST(population AS Float)/CAST(area AS Float) <= (SELECT AVG(CAST(population AS Float)/CAST(area AS Float)) FROM facts);

 * sqlite:///factbook.db
Done.


Countries with density below or avarage
211


In [19]:
%%sql

SELECT COUNT(*) 'Countries with density above avarage'
    FROM facts
WHERE CAST(population AS Float)/CAST(area AS Float) > (SELECT AVG(CAST(population AS Float)/CAST(area AS Float)) FROM facts);

 * sqlite:///factbook.db
Done.


Countries with density above avarage
25


In fact, we see the countries below avarage are ten times more numerous than the above avarage.